In [67]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

In [2]:
#Question 1
train_data_name=input("Enter the name of train data file [(ex) harris.dat] : ") # data name
test_data_name=input("Enter the name of test data file [(ex) harris.dat] : ") 
coding_fm=int(input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")) # data separator
separator_fm={coding_fm ==1 : " "}.get(True, ",")
res_pos=int(input("Enter the column position of the response variable : [from 1 to p] : "))
header=input("Does the data have column header? (y/n) : ")
if(header=="y") : trdata=pd.read_csv(train_data_name, sep=separator_fm) # loading data
else : trdata=pd.read_csv(train_data_name, sep=separator_fm, header=None) # loading data
if(header=="y") : tstdata=pd.read_csv(test_data_name, sep=separator_fm) # loading data
else : tstdata=pd.read_csv(test_data_name, sep=separator_fm, header=None) # loading data
out_name=input("Enter the output file name to export [(ex) result.txt] : ")

# Extracting X and y
data = pd.DataFrame(trdata)
C = pd.DataFrame(np.ones(shape = (data.shape[0],)))
data = pd.concat([C,data], axis = 1, ignore_index= True)
X= data.drop([data.columns[res_pos]], axis=1 )
Y= data.iloc[:, res_pos ].values

#Extracting test X and y
tstdata = pd.DataFrame(tstdata)
tstC = pd.DataFrame(np.ones(shape = (tstdata.shape[0],)))
tstdata = pd.concat([tstC,tstdata], axis = 1, ignore_index= True)
tstX= tstdata.drop([tstdata.columns[res_pos]], axis=1 )
tstY= tstdata.iloc[:, res_pos ].values

Enter the name of train data file [(ex) harris.dat] : pid.dat
Enter the name of test data file [(ex) harris.dat] : pidtest.dat
Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): 2
Enter the column position of the response variable : [from 1 to p] : 8
Does the data have column header? (y/n) : n
Enter the output file name to export [(ex) result.txt] : result.txt


In [3]:
def gini(Y):
  Y_1 =0; Y_2=0
  Z = np.unique(Y)
  for i in range(0,len(Y)):
    if Y[i] == Z[0]:
      Y_1 +=1
    else:
      Y_2 +=1 
  p1 = Y_1 / len(Y)
  p2 = Y_2 / len(Y)
  gini = 1- p1**2 -p2**2
  
  return gini


In [5]:
def one_level_decision_tree_fit(X,Y):
  A=[]
  B = np.unique(Y)
  for i in range(1,len(X.iloc[0])):
    A.append(len(X.iloc[:,i].sort_values().unique()))
  c = np.max(A)
  K = pd.DataFrame(index = range(1,len(X.iloc[0])), columns = range(0,c))
  for i in range(1,len(X.iloc[0])):
    Z=[]
   
    for j in range(0,len(X.iloc[:,i].sort_values().unique())-1):
      z1 = X.iloc[:,i].sort_values().unique()[j]
      z2 = X.iloc[:,i].sort_values().unique()[j+1]
      Z.append((z1+z2)/2)
      print(f'{i}', f', {j}')
      for k in range(0,len(Z)):
        condition = X.iloc[:,i]<=Z[k]
        y1 =0; y2=0; y3=0 ;y4=0;
        
        for s in range(0,len(Y[condition==True])):
          if Y[condition==True][s] == 1:
            y1 += 1
          else:
            y2 += 1
        T = y1 / (y1+y2)

        for s in range(0,len(Y[condition==False])):
          if Y[condition==False][s] == 2:
            y3 += 1
          else:
            y4 += 1
        P = y3 / (y3+y4)

        p1 = 1-T**2 - (1-T)**2 
        p2 = 1-P**2 - (1-P)**2

        g = (len(Y[condition==True])/len(Y)*p1) + ((len(Y[condition==False])/len(Y))*p2)

        s = gini(Y) - g
        K[k][i] = s

  K = K.astype(float)
  idx = []
  col = K.idxmax(axis=1)
  for i in range(1,len(X.iloc[0])):
    idx.append(K[col[i]][i])
  idx_r = np.argmax(idx) # 첫 노드를 나누는 변수
  idx_c = col[idx_r] 
  
  Z=[]
  for j in range(0,len(X.iloc[:,idx_r].sort_values().unique())-1):
    z1 = X.iloc[:,idx_r].sort_values().unique()[j]
    z2 = X.iloc[:,idx_r].sort_values().unique()[j+1]
    Z.append((z1+z2)/2)
  l = Z[idx_c] # 임계점
  
  condition = X.iloc[:,idx_r]<=l
  y1 =0; y2=0; y3=0 ;y4=0;
  B = np.unique(Y)
  for s in range(0,len(Y[condition==True])):
    if Y[condition==True][s] == 1:
      y1 += 1
    else:
      y2 += 1
  T = y1 / (y1+y2)

  for s in range(0,len(Y[condition==False])):
    if Y[condition==False][s] == 2:
      y3 += 1
    else:
      y4 += 1
  P = y3 / (y3+y4)

  print('tree structure')
  print(f'Node 1 : x{idx_r} <= {l}', f'( {len(Y[Y==B[0]])}, {len(Y[Y==B[1]])})')
  if T>=P:
    print('Node 2: 1', f'({y1}.{y2})')
    print('Node 3: 2', f'({y3},{y4})')
  if T<P: 
    print('Node 2: 2', f'({y1}.{y2})')
    print('Node 3: 1', f'({y3},{y4})')

  return idx_r, l



In [ ]:
K = one_level_decision_tree_fit(X,Y)

In [ ]:
idx_r = K[0]
l = K[1]

In [88]:
def one_level_decision_tree(X,Y):
  Z=[]
  for j in range(0,len(X.iloc[:,idx_r].sort_values().unique())-1):
    z1 = X.iloc[:,idx_r].sort_values().unique()[j]
    z2 = X.iloc[:,idx_r].sort_values().unique()[j+1]
    Z.append((z1+z2)/2)
  l = Z[idx_c]
  
  condition = X.iloc[:,idx_r]<=l
  y1 =0; y2=0; y3=0 ;y4=0;
  B = np.unique(Y)
  for s in range(0,len(Y[condition==True])):
    if Y[condition==True][s] == 1:
      y1 += 1
    else:
      y2 += 1
  T = y1 / (y1+y2)

  for s in range(0,len(Y[condition==False])):
    if Y[condition==False][s] == 2:
      y3 += 1
    else:
      y4 += 1
  P = y3 / (y3+y4)
  Y2=Y
  if T>=P :
    Y2[condition==True] =1
    Y2[condition==False] = 0
  else: 
    Y2[condition==True] = 0
    Y2[condition==False] = 1

  confusion_tst = confusion_matrix(Y, Y2)

  accu_tst = 0
  for i in range(len(np.unique(Y))):
    accu_tst = accu_tst + confusion_tst[i][i]
  accuracy_tst = accu_tst / X.shape[0]

  print('\n\nconfusion matrix (test)')
  print('----------------------------------')

  print('           predicted class \n Actual 1 ' ,confusion_tst[0], '\n class  2 ', confusion_tst[1])
  for i in range(2,len(np.unique(Y)) ):
    print(f'        {i+1} ', confusion_tst[i])
  print('model summary')
  print('----------------------------------')
  print('Overall accuracy = ' ,accuracy_tst)
    

In [89]:
 one_level_decision_tree(tstX,tstY)
  



confusion matrix (test)
----------------------------------
           predicted class 
 Actual 1  [47  0] 
 class  2  [  0 179]
model summary
----------------------------------
Overall accuracy =  1.0
